In [43]:
import requests
import json
import uuid

# Configuração
BASE_URL_ADK_SERVER = "https://adk-server-health-894271896157.europe-west1.run.app"
BASE_URL_FASTAPI = "https://fastapi-server-health-894271896157.europe-west1.run.app"


def get_all_apps(base_url=BASE_URL_ADK_SERVER):
    """
    Recupera a lista de todos os aplicativos disponíveis na API ADK.
    
    Returns:
        list - Lista de aplicativos.
    """
    response = requests.get(f"{base_url}/list-apps")
    return response.json()

all_apps = get_all_apps()
all_apps

['compliance_agent',
 'parallel_analyzer_agent',
 'search_agent',
 'sequential_analyzer_agent',
 'simple_prescription_agent']

In [44]:
# Load data 
with open("../data/patients_ready_to_dashboard.json", "r", encoding="utf-8") as f:
    df_ready_to_dashboard = json.load(f)

In [46]:
sample = df_ready_to_dashboard[:10]

In [47]:
inputs_to_llm = [patient['admission_str_input_to_llm'] for patient in sample]

In [48]:
inputs_to_llm

["Subject ID: 0\nAdmission ID: 169938\nAdmission Date: 4323585540000\nDischarge Date: 4324203900000\nInsurance: Medicare\nAge: None\nGender: M\nLanguage: None\nMarital Status: UNKNOWN (DEFAULT)\nEthnicity: WHITE\nCreatinine: 1.1 mg/dL\nHemoglobin: 10.0 g/dL\nPotassium: 3.7 mEq/L\nSodium: 142.0 mEq/L\nNotes (24h Summary): None\nPrescriptions:   - Drug: SW, Type: BASE, Dose: 100 ml, Form: None, Route: IV, Start: 2107-01-06 00:00:00, Stop: 2107-01-07 00:00:00--------------------------------\n  - Drug: NS, Type: BASE, Dose: 500 ml, Form: None, Route: IV, Start: 2107-01-06 00:00:00, Stop: 2107-01-07 00:00:00--------------------------------\n  - Drug: NS, Type: BASE, Dose: 500 ml, Form: None, Route: IV, Start: 2107-01-06 00:00:00, Stop: 2107-01-07 00:00:00--------------------------------\n  - Drug: Iso-Osmotic Dextrose, Type: BASE, Dose: 200 ml, Form: None, Route: IV, Start: 2107-01-07 00:00:00, Stop: 2107-01-08 00:00:00--------------------------------\n  - Drug: Fat Emulsion 20%, Type: BASE

In [49]:
# Simple API wrapper functions
def analyze_simple(health_data: str):
    """Simple prescription analysis"""
    response = requests.post(
        f"{BASE_URL_FASTAPI}/analyze/simple",
        json={"health_data": health_data}
    )
    return response.json()

def analyze_parallel(health_data: str):
    """Parallel multi-agent analysis"""
    response = requests.post(
        f"{BASE_URL_FASTAPI}/analyze/parallel", 
        json={"health_data": health_data}
    )
    return response.json()

def analyze_sequential(health_data: str):
    """Sequential health analysis"""
    response = requests.post(
        f"{BASE_URL_FASTAPI}/analyze/sequential",
        json={"health_data": health_data}
    )
    return response.json()

def analyze_all(health_data: str):
    """All three analyses at once"""
    response = requests.post(
        f"{BASE_URL_FASTAPI}/analyze/all",
        json={"health_data": health_data}
    )
    return response.json()

print("API wrapper functions loaded:")
print("- analyze_simple(health_data)")
print("- analyze_parallel(health_data)")
print("- analyze_sequential(health_data)")
print("- analyze_all(health_data)")

API wrapper functions loaded:
- analyze_simple(health_data)
- analyze_parallel(health_data)
- analyze_sequential(health_data)
- analyze_all(health_data)


In [51]:
# Test with first patient
if inputs_to_llm:
    patient_data = inputs_to_llm[0]
    
    # Test simple analysis
    result = analyze_simple(patient_data)
    print("Simple analysis result:")
    print(json.dumps(result, indent=2))
else:
    print("No patient data available for testing")

Simple analysis result:
{
  "status": "success",
  "data": {
    "state": {},
    "user_id": "api_user",
    "session_id": "s_f1ec0967"
  },
  "message": "Simple prescription analysis completed successfully"
}


In [52]:
result

{'status': 'success',
 'data': {'state': {}, 'user_id': 'api_user', 'session_id': 's_f1ec0967'},
 'message': 'Simple prescription analysis completed successfully'}

In [53]:
inputs_to_llm[0]

"Subject ID: 0\nAdmission ID: 169938\nAdmission Date: 4323585540000\nDischarge Date: 4324203900000\nInsurance: Medicare\nAge: None\nGender: M\nLanguage: None\nMarital Status: UNKNOWN (DEFAULT)\nEthnicity: WHITE\nCreatinine: 1.1 mg/dL\nHemoglobin: 10.0 g/dL\nPotassium: 3.7 mEq/L\nSodium: 142.0 mEq/L\nNotes (24h Summary): None\nPrescriptions:   - Drug: SW, Type: BASE, Dose: 100 ml, Form: None, Route: IV, Start: 2107-01-06 00:00:00, Stop: 2107-01-07 00:00:00--------------------------------\n  - Drug: NS, Type: BASE, Dose: 500 ml, Form: None, Route: IV, Start: 2107-01-06 00:00:00, Stop: 2107-01-07 00:00:00--------------------------------\n  - Drug: NS, Type: BASE, Dose: 500 ml, Form: None, Route: IV, Start: 2107-01-06 00:00:00, Stop: 2107-01-07 00:00:00--------------------------------\n  - Drug: Iso-Osmotic Dextrose, Type: BASE, Dose: 200 ml, Form: None, Route: IV, Start: 2107-01-07 00:00:00, Stop: 2107-01-08 00:00:00--------------------------------\n  - Drug: Fat Emulsion 20%, Type: BASE,